This notebook is based on the detailed guide [here](https://dagshub.com/blog/how-to-install-kubeflow-locally/).


Prerequisite:
- kubectl
- kustomize
- minikube



First, Clone the manifests repo from the Kubeflow team:


In [ ]:
!git clone https://github.com/kubeflow/manifests.git

## Minikube Cluster
Let's start a k8s cluster locally with minikube.

> Note: At the time of writing, Kubeflow 1.7(latest) only validated with k8s version 1.25/1.24, while minikube creates kubernetes at v1.27. There is error related to HorizontalPodAutoscaler during resource applying. You might see the issue [here](https://stackoverflow.com/questions/76502195/horizontalpodautoscaler-not-found-on-minikube-when-installing-kubeflow). As a result, v1.25 is used here.

In [2]:
!minikube start --kubernetes-version=v1.25.0

😄  minikube v1.30.1 on Darwin 12.1
❗  Kubernetes 1.25.0 has a known issue with resolv.conf. minikube is using a workaround that should work for most use cases.
❗  For more information, see: https://github.com/kubernetes/kubernetes/issues/112135
✨  Automatically selected the docker driver
📌  Using Docker Desktop driver with root privileges
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
💾  Downloading Kubernetes v1.25.0 preload ...
    > preloaded-images-k8s-v18-v1...:  385.37 MiB / 385.37 MiB  100.00% 3.04 Mi
🔥  Creating docker container (CPUs=2, Memory=7911MB) ...
🐳  Preparing Kubernetes v1.25.0 on Docker 23.0.2 ...
    ▪ Generating certificates and keys ...
    ▪ Booting up control plane ...
    ▪ Configuring RBAC rules ...
🔗  Configuring bridge CNI (Container Networking Interface) ...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🔎  Verifying Kubernetes components...
🌟  Enabled addons: storage-provisioner, default-storageclass

❗ 

In [1]:
!minikube kubectl version -- --short

Flag --short has been deprecated, and will be removed in the future. The --short output will become the default.
Client Version: v1.25.0
Kustomize Version: v4.5.7
The connection to the server 127.0.0.1:65526 was refused - did you specify the right host or port?


Optional - You can open the dashboard by running :
```shell
minikube dashboard
```

And check your cluster status:

In [2]:
!minikube status

minikube
type: Control Plane
host: Stopped
kubelet: Stopped
apiserver: Stopped
kubeconfig: Stopped



## Launch Kubeflow

We need to install kubeflow via deploying via the manifest we cloned by the commands early on.
This is a simple single command to install kubeflow, refer to [here](https://github.com/kubeflow/manifests#install-with-a-single-command).

> If problem happens or it takes too long, check again kubectl version and minikube version.

In [8]:
!cd manifests; while ! kustomize build example | awk '!/well-defined/' | kubectl apply -f -; do echo "Retrying to apply resources"; sleep 10; done

# Warning: 'patchesStrategicMerge' is deprecated. Please use 'patches' instead. Run 'kustomize edit fix' to update your Kustomization automatically.
# Warning: 'vars' is deprecated. Please use 'replacements' instead. [EXPERIMENTAL] Run 'kustomize edit fix' to update your Kustomization automatically.
# Warning: 'vars' is deprecated. Please use 'replacements' instead. [EXPERIMENTAL] Run 'kustomize edit fix' to update your Kustomization automatically.
# Warning: 'patchesStrategicMerge' is deprecated. Please use 'patches' instead. Run 'kustomize edit fix' to update your Kustomization automatically.
# Warning: 'patchesStrategicMerge' is deprecated. Please use 'patches' instead. Run 'kustomize edit fix' to update your Kustomization automatically.
# Warning: 'patchesStrategicMerge' is deprecated. Please use 'patches' instead. Run 'kustomize edit fix' to update your Kustomization automatically.
# Warning: 'bases' is deprecated. Please use 'resources' instead. Run 'kustomize edit fix' to update

Run below command to wait for all condition being ready.

In [6]:
!kubectl wait --for=condition=Ready pods --all --all-namespaces

pod/dex-56d9748f89-r8zrn condition met
pod/cert-manager-74d949c895-pvfj6 condition met
pod/cert-manager-cainjector-d9bc5979d-wkrcc condition met
pod/cert-manager-webhook-84b7ddd796-kq6tw condition met
pod/authservice-0 condition met
pod/cluster-local-gateway-6955b67f54-fmrg5 condition met
pod/istio-ingressgateway-67f7b5f88d-ldz7n condition met
pod/istiod-56f7cf9bd6-kmr4m condition met
pod/eventing-controller-c6f5fd6cd-vlrlc condition met
pod/eventing-webhook-79cd6767-fdhbx condition met
pod/activator-67849589d6-ndxnw condition met
pod/autoscaler-6dbcdd95c7-ljwq8 condition met
pod/controller-b9b8855b8-tfrwd condition met
pod/domain-mapping-75cc6d667f-2ggds condition met
pod/domainmapping-webhook-6dfb78c944-8fnzd condition met
pod/net-istio-controller-5fcd96d76f-b62jm condition met
pod/net-istio-webhook-7ff9fdf999-8xzzp condition met
pod/webhook-69cc5b9849-t5ssc condition met
pod/coredns-565d847f94-l2rmj condition met
pod/etcd-minikube condition met
pod/kube-apiserver-minikube condition 

### [Optional] Resume the stopped cluster container
If we stopped the cluster container and wanted to resume it, we can directly run below command to restart it.
```bash
minikube start
```



Almost there! Now we just need to setup the port forwarding to allow access via locahost.
Copy below command and run in a terminal to setup the ingress gateway to listen to port `8080` from local, and forward it to port `80`.

> Note: kubectl port-forward does not return. To continue with the exercises, you will need to open another terminal.

```bash
kubectl port-forward svc/istio-ingressgateway -n istio-system 8080:80
```

Now the dashboard is ready on [http://localhost:8080/](http://localhost:8080/). You can read the official [kubeflow doc](https://www.kubeflow.org/docs/components/) to experiment on kubeflow now!

-------

## Clean up

Run below command to stop minikube

In [ ]:
!minikube stop

Or remove your **cluster** by :

In [ ]:
!minikube delete